
# Contrastive learning as an instance discriminator with a rank metric

This file contains code to demonstrate how contrastive learning is effectively an instance discrimination problem.
In particular, we present a similarity rank based metric that can determine the difficulty of the instance discrimination task.

The metric is computed as follows:
1) The similarity matrix for each batch is ranked row wise. In our setup, the highest similarity value between the representation of the two augmentations is the largest rank.

2) Compute the trace of the ranked simialrity matrix. Since, diagonal of the similarity matrix refers to a positive pair, for better learning we would expect the diagonal values of the rank matrix to be higher. 

3) Compute Trace/batchsize which gives average rank of a positive pair.

We use openly available public dataset [Nomao](https://archive.ics.uci.edu/dataset/227/nomao) and are interested in learning contrastive loss based data representation.

In [112]:
# importing packages

import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm, linear_model, model_selection, metrics, ensemble
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score, average_precision_score, pairwise,mutual_info_score,mean_squared_error
from scipy import linalg, stats
import os.path
import pandas as pd
import matplotlib.pyplot as plt
import random
from matching.games import StableMarriage
import pingouin as pg
import datetime
# from datetime import datetime
import json, sys, argparse
from tqdm.auto import tqdm
import pandas as pd

# import requests
# from transformers import CLIPProcessor, CLIPModel
# from PIL import Image
# from datasets import load_dataset
# import torchvision.datasets as dset
# import torchvision.transforms as transforms
# import base64
# import io
# from urllib.request import urlopen

In [113]:
# function for computing the contrastive loss and the similarity metric
def NTXentLoss(z1, z2,
                 temperature=0.1): 
    # compute the cosine similarity but first normalizing and then matrix multiplying the known and unknown tensors
    cos_sim_o = torch.div(torch.matmul(torch.nn.functional.normalize(z1),
                                       torch.transpose(torch.nn.functional.normalize(z2), 0, 1)),
                          temperature)

    # for numerical stability  ## TODO update this logit name
    logits_max_o, _ = torch.max(cos_sim_o, dim=1, keepdim=True)
    logits_o = cos_sim_o - logits_max_o.detach()

    # breakpoint()
    if True:
      # computing the exp logits
      exp_o = torch.exp(logits_o)
      batch_loss_o = - torch.log(exp_o.diag() / exp_o.sum(dim=0)).sum() - torch.log(
        exp_o.diag() / exp_o.sum(dim=1)).sum()
      # computing the avg rank of the positive examples for checking if the algo is learning the representation closer
      # since we are computing the rank on the similarity so higher the better
      avg_rank_cos_sim_o = np.trace(stats.rankdata(cos_sim_o.cpu().detach().numpy(), axis=1)) / len(cos_sim_o)

    # print("This batch's loss and avg rank ", batch_loss_o.item(), batch_loss_r.item(), avg_rank_cos_sim_o, avg_rank_cos_sim_r)
    return batch_loss_o, avg_rank_cos_sim_o

In [114]:
# function for initial loading and processing of the dataset
def load_dataset(dataset):
    if dataset=='SD_2':
        filename = "2021-05-18Syn_Data_2_Sample_no_1_size_20_10000_for_AE_balanced.csv" 
        data_file = os.path.join('./Small_datasets', dataset, filename)
        full_Data0 = pd.read_csv(data_file)
        
    if dataset=='Nomao':
        data_file = os.path.join('./Small_datasets', dataset, dataset + ".data")

        full_Data0 = pd.read_csv(data_file, header=None)
        full_Data0.replace('?', np.nan, inplace=True)
        full_Data0 = full_Data0.loc[:, ~(full_Data0 == 'n').any()]  # dropping the nominal type columns
        full_Data0.drop(columns=[0], inplace=True)  # dropping the id column

        # drop columns with very high missing percentage
        percent_missing = full_Data0.isnull().sum() * 100 / len(full_Data0)
        missing_value_df = pd.DataFrame({'column_name': full_Data0.columns,
                                         'percent_missing': percent_missing}).sort_values(by='percent_missing')
        full_Data0 = full_Data0[missing_value_df[missing_value_df['percent_missing'] < 70.0]['column_name']]

        ## final columns that were selected including the label and excluding the first column that was the id
        ## [1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 49, 50, 51, 52, 53, 54, 57, 58, 59, 60, 61, 62, 65, 66, 67, 68, 69,
        # 70, 73, 74, 75, 76, 77, 78, 81, 82, 83, 84, 85, 86, 89, 90, 91, 97, 98, 99, 101, 102, 103, 105, 106, 107, 109,
        # 110, 111, 113, 114, 115, 117, 118, 119]

        full_Data0 = full_Data0.reindex(columns=sorted(full_Data0.columns))  # reindexing so that label is at the end
        new_feature_names = ['Col' + str(i + 1) for i in range(full_Data0.shape[1] - 1)] 
        full_Data0 = full_Data0.set_axis(new_feature_names+ ['Y'], axis=1)  # renamed the continuous columns

        # converting the columns that are being treated as object type but actually are float
        for a in full_Data0.columns:
            if full_Data0[a].dtype == 'object':
                full_Data0[a] = full_Data0[a].astype('float')

        full_Data0 = full_Data0.fillna(full_Data0.mean())  # filling the missing values
    
    full_Data = full_Data0.copy()
    full_Data['Y'] = np.where(full_Data['Y'] == -1, 0, 1)  # to make it compatible with the rest    
    num_sample = full_Data.shape[0]
    num_features = full_Data.shape[1] - 1

    # full data initial correlation
    Feature_matrix = full_Data.iloc[:, :-1]
    Cor_from_df = Feature_matrix.corr()
    
    return full_Data, num_sample, num_features, Cor_from_df

In [115]:
class TabularDataset(Dataset):
    def __init__(self, data, output_col=None):
        """
        Characterizes a Dataset for PyTorch

        Parameters
        ----------

        data: pandas data frame
          The data frame object for the input data. It must
          contain all the continuous, categorical and the
          output columns to be used.

        cat_cols: List of strings
          The names of the categorical columns in the data.
          These columns will be passed through the embedding
          layers in the model. These columns must be
          label encoded beforehand.

        output_col: string
          The name of the output variable column in the data
          provided.
        """

        self.n = data.shape[0]

        if output_col:
            self.y = data[output_col].astype(np.float32).values.reshape(-1, 1)
        else:
            self.y = np.zeros((self.n, 1))

        # self.cat_cols = cat_cols if cat_cols else []
        self.cont_cols = [col for col in data.columns
                          if col not in [output_col]]
        # print(self.cont_cols)

        if self.cont_cols:
            self.cont_X = data[self.cont_cols].astype(np.float32).values
        else:
            self.cont_X = np.zeros((self.n, 1))
    def __len__(self):
        """
        Denotes the total number of samples.
        """
        return self.n

    def __getitem__(self, idx):
        """
        Generates one sample of data.
        """
        return [self.y[idx], self.cont_X[idx]]

In [116]:
def generate_noisy_xbar(x, noise_type="Zero-out", noise_level=0.1):
  """Generates noisy version of the samples x; Noise types: Zero-out, Gaussian, or Swap noise

  Args:
      x (np.ndarray): Input data to add noise to

  Returns:
      (np.ndarray): Corrupted version of input x

  """
  # Dimensions
  no, dim = x.shape
  # Initialize corruption array
  x_bar = torch.zeros_like(x)

  # Randomly (and column-wise) shuffle data
  if noise_type == "swap_noise":
    for i in range(dim):
      idx = torch.randperm(no)
      x_bar[:, i] = x[idx, i]
  # Elif, overwrite x_bar by adding Gaussian noise to x
  elif noise_type == "gaussian_noise":
    # breakpoint()
    x_bar = x + torch.normal(0, noise_level, size=x.shape, device='cuda')
  else:
    x_bar = x_bar

  return x_bar

In [117]:
# encoder networks
class MLP(torch.nn.Sequential):
    """Simple multi-layer perceptron with ReLu activation and optional dropout layer"""

    def __init__(self, input_dim, hidden_dim, n_layers, dropout=0.0):
        layers = []
        in_dim = input_dim
        for _ in range(n_layers - 1):
            layers.append(torch.nn.Linear(in_dim, hidden_dim))
            layers.append(nn.ReLU(inplace=True))
            layers.append(torch.nn.Dropout(dropout))
            in_dim = hidden_dim

        layers.append(torch.nn.Linear(in_dim, hidden_dim))

        super().__init__(*layers)
        
class CL_model(nn.Module):
    def __init__(
        self,
        input_dim,
        emb_dim,
        encoder_depth=4,
        head_depth=2,
    ):
        """Implementation of a SimCLR kind basic CL approach.
        It consists of an encoder that learns the embeddings.
        It is done by minimizing the contrastive loss of a sample and an augmented view of it.
            Args:
                input_dim (int): size of the inputs
                emb_dim (int): dimension of the embedding space
                encoder_depth (int, optional): number of layers of the encoder MLP. Defaults to 4.
                head_depth (int, optional): number of layers of the pretraining head. Defaults to 2.
        """
        super().__init__()


        self.encoder = MLP(input_dim, emb_dim, encoder_depth)
        self.pretraining_head = MLP(emb_dim, emb_dim, head_depth)

        # initialize weights
        self.encoder.apply(self._init_weights)
        self.pretraining_head.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            module.bias.data.fill_(0.01)
            
    def forward(self,x):
        rep = self.encoder(x)
        proj = self.pretraining_head(rep)
        return rep, proj

In [118]:
# initial setting and the dataset choice

dataset = "Nomao" # dataset name  {'SD_2', 'Nomao'}
batchSize = 64
learningRate = 0.0004
LRPatience = 5
learningRateFactor=0.3931
dropout_rate_CL = 0.1278
epochs = 50
masking_ratio = 0.1306
tau = 0.1893
randomSeed = 784  # random seed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
enc_depth = 5
proj_depth = 4
Embedding_dim= 15
outcome = 'Y'

saving_dir = '/home/trips/ContrastiveLearning_Tutorial/Rank_Figures/'
if not os.path.exists(saving_dir):
    os.makedirs(saving_dir)

In [119]:
# loading data
full_data, num_sample, num_features, Cor_from_df = load_dataset(dataset)

# data dimensions 
print(" Original data size :  ", full_data.shape)  # Number of samples *  number of features

/tmp/ipykernel_3027269/2349951105.py:11: DtypeWarning: Columns (57,58,59,60,61,62,65,66,67,68,69,70,101,102,103,105,106,107,117) have mixed types. Specify dtype option on import or set low_memory=False.
  full_Data0 = pd.read_csv(data_file, header=None)


 Original data size :   (34465, 63)


In [120]:
# keeping a holdout sample aside
Df_for_training, Df_holdout = model_selection.train_test_split(full_data, test_size=0.2,
                                                        random_state=42,
                                                        stratify=full_data[outcome])

## Training prep

In [121]:
#explicitly setting the nworkers in the dataloader was incompatible with the jupyter kernel
dataset_orig = TabularDataset(data=Df_for_training, output_col=outcome)
train_loader_orig = DataLoader(dataset_orig, batchSize, shuffle=True)
dataset_orig_val = TabularDataset(data=Df_holdout, output_col=outcome)
val_loader_orig = DataLoader(dataset_orig_val, batchSize, shuffle=True)

In [122]:
model = CL_model(input_dim=num_features,emb_dim=Embedding_dim).to(device)

optimizer = optim.Adam(model.parameters(), lr=learningRate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=LRPatience, verbose=True)

## Training

In [ ]:
total_perepoch_loss = []
total_perepoch_rank = []
for epoch in range(epochs):
    loss_tr_o = 0
    rank_tr_o = 0
    counting_flag_for_rank = 0
    # computing the representations from the trained encoders
    model.train()

    for i, data in enumerate(train_loader_orig):
        x = data[1].to(device)
        x_bar = x
        x_bar_noisy = generate_noisy_xbar(x_bar)
        # Generate binary mask
        mask = torch.tensor(np.random.binomial(1, masking_ratio, x_bar.shape)).to(device)
        mask1 = torch.tensor(np.random.binomial(1, masking_ratio, x_bar.shape)).to(device)
        # breakpoint()
        # Replace selected x_bar features with the noisy ones
        x_aug1 = x_bar * (1 - mask) + x_bar_noisy * mask
        x_aug2 = x_bar * (1 - mask1) + x_bar_noisy * mask1
    
#         print(x_aug1.shape)
    
        _, rep1 = model(x_aug1)
        _, rep2 = model(x_aug2)
        contrastive_loss_o, avg_Rank_o = NTXentLoss(rep1, rep2, temperature=tau)
    
        # compute accumulated gradients
        contrastive_loss_o.backward()
    
        # perform parameter update based on current gradients
        optimizer.step()
    
        # add the mini-batch training loss to epoch loss
        loss_tr_o += contrastive_loss_o.item()
        if (data[1].shape[0] == batchSize):
            rank_tr_o += avg_Rank_o.item()
            counting_flag_for_rank = counting_flag_for_rank + 1
    
        # compute the epoch training loss
    loss_tr_o = loss_tr_o / (len(train_loader_orig))
    rank_tr_o = rank_tr_o / (
        counting_flag_for_rank)  # dividing by counting_flag_for_rank because the avg rank from all batches in not included

    with torch.no_grad():
        # computing the representations from the trained encoders
        model.eval()
        loss_val_o = 0
        rank_val_o = 0
        counting_flag_for_rank_val = 0
    
        for i, data in enumerate(val_loader_orig):
            x = data[1].to(device)
            x_bar = x
            x_bar_noisy = generate_noisy_xbar(x_bar)
            # Generate binary mask
            mask = torch.tensor(np.random.binomial(1, masking_ratio, x_bar.shape)).to(device)
            mask1 = torch.tensor(np.random.binomial(1, masking_ratio, x_bar.shape)).to(device)
            # breakpoint()
            # Replace selected x_bar features with the noisy ones
            x_aug1 = x_bar * (1 - mask) + x_bar_noisy * mask
            x_aug2 = x_bar * (1 - mask1) + x_bar_noisy * mask1
    
            _, rep1 = model(x_aug1)
            _, rep2 = model(x_aug2)
            contrastive_loss_o, avg_Rank_o = NTXentLoss(rep1, rep2, temperature=tau)
    
            # add the mini-batch training loss to epoch loss
            loss_val_o += contrastive_loss_o.item()
            if (data[1].shape[0] == batchSize):
                rank_val_o += avg_Rank_o.item()
                counting_flag_for_rank_val = counting_flag_for_rank_val + 1
            
    
        # compute the epoch training loss
        loss_val_o = loss_val_o / (len(val_loader_orig))
        rank_val_o = rank_val_o / (counting_flag_for_rank_val)
        total_perepoch_loss.append(loss_val_o)
        total_perepoch_rank.append(rank_val_o)
    
        # display the epoch validation loss
        print("Validation performance epoch : {}/{}, loss_o = {:.5f},  avgbatchwise_rank_o = {:.5f}".format(
            epoch + 1, epochs, loss_val_o, rank_val_o))
    scheduler.step(loss_val_o)

Validation performance epoch : 1/50, loss_o = 434.96180,  avgbatchwise_rank_o = 52.62734
Validation performance epoch : 2/50, loss_o = 431.55502,  avgbatchwise_rank_o = 53.18721
Validation performance epoch : 3/50, loss_o = 394.86503,  avgbatchwise_rank_o = 55.90859
Validation performance epoch : 4/50, loss_o = 415.02502,  avgbatchwise_rank_o = 55.25226
Validation performance epoch : 5/50, loss_o = 388.18207,  avgbatchwise_rank_o = 56.64413
Validation performance epoch : 6/50, loss_o = 366.42929,  avgbatchwise_rank_o = 57.66443
Validation performance epoch : 7/50, loss_o = 369.88381,  avgbatchwise_rank_o = 57.59273
Validation performance epoch : 8/50, loss_o = 358.68931,  avgbatchwise_rank_o = 57.79629
Validation performance epoch : 9/50, loss_o = 365.44922,  avgbatchwise_rank_o = 57.80345
Validation performance epoch : 10/50, loss_o = 360.01618,  avgbatchwise_rank_o = 57.98058
Validation performance epoch : 11/50, loss_o = 349.19862,  avgbatchwise_rank_o = 58.12383
Validation performa

## Loss and Rank metric visualization

In [ ]:
# PLotting the loss

plt.figure()
plt.plot(np.arange(len(total_perepoch_loss)), total_perepoch_loss)
plt.xlabel(" epochs ")
plt.ylabel(" Contrastive loss for the epoch ")
plt.title(" Contrastive loss versus epochs")
plt.legend()
plt.show()

In [ ]:
plt.savefig(
    saving_dir +  "/Contrastive_loss_prog_" + str(dataset) + "Emb_dim" + str(
        Embedding_dim) + "_tau_" + str(tau) + "_batchsize_" + str(
        batchSize) + "_epochs_" + str(epochs) +  ".png")
plt.savefig(
    saving_dir +  "/Contrastive_loss_prog_" + str(dataset) + "Emb_dim" + str(
        Embedding_dim) + "_tau_" + str(tau) + "_batchsize_" + str(
        batchSize) + "_epochs_" + str(epochs) +  ".pdf")
plt.close()

In [ ]:
# plotting the rank metric

plt.figure()
plt.plot(np.arange(len(total_perepoch_rank)), total_perepoch_rank)
plt.xlabel(" epochs ")
plt.ylabel(" Rank metric for the epoch ")
plt.title(" Rank metric when the batch size is " + str(batchSize) + "\n Best rank value is batch size")
# plt.legend()
plt.show()

In [ ]:
plt.savefig(
    saving_dir +  "/Rank_metric_prog_" + str(dataset) + "Emb_dim" + str(
        Embedding_dim) + "_tau_" + str(tau) + "_batchsize_" + str(
        batchSize) + "_epochs_" + str(epochs) +  ".png")
plt.savefig(
    saving_dir +  "/Rank_metric_prog_" + str(dataset) + "Emb_dim" + str(
        Embedding_dim) + "_tau_" + str(tau) + "_batchsize_" + str(
        batchSize) + "_epochs_" + str(epochs) +  ".pdf")
plt.close()